# Songchai Duangpan ID: 6136896

# เรียก lib ที่เป็น Deep Learning Model ซึ่งเราจะใช้ Keras

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
import numpy as np
import cv2
import os
import imutils

In [18]:
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import to_categorical
from keras.layers import Dropout

## เรียก lib สำหรับ Evaluate โมเดล ##

In [3]:
from sklearn.metrics import classification_report 
from sklearn.metrics import f1_score ,accuracy_score

In [4]:
def image_to_feature_vector(image, size=(32, 32)):
    return cv2.resize(image, size).flatten()

## สร้างฟังก์ชั่นขึ้นมาหนึ่งอันเพื่อแปลงรูป ให้เป็น Histogram

In [5]:
def extract_color_histogram(image, bins=(8, 8, 8)):
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    hist = cv2.calcHist([hsv], [0, 1, 2], None, bins,
        [0, 180, 0, 256, 0, 256])

    if imutils.is_cv2():
        hist = cv2.normalize(hist)
 
    else:
        cv2.normalize(hist, hist)
 

    return hist.flatten()

In [6]:
rawImages = []
labels = []

In [7]:
from os import listdir
from os.path import isfile, join
mypath = 'train/'
imagePaths = [mypath+f for f in listdir(mypath) if isfile(join(mypath, f))]

# เรียกฟังก์ชั่น Histogram เพื่อจัดการกัน

In [8]:
for (i, imagePath) in enumerate(imagePaths):
    image = cv2.imread(imagePath)
    label = imagePath.split(os.path.sep)[-1].split(".")[0]

    #pixels = image_to_feature_vector(image)
    pixels = extract_color_histogram(image)
    
    rawImages.append(pixels)
    labels.append(label)
    
    if i > 0 and i % 1000 == 0:
        print("[INFO] processed {}/{}".format(i, len(imagePaths)))

[INFO] processed 1000/25000
[INFO] processed 2000/25000
[INFO] processed 3000/25000
[INFO] processed 4000/25000
[INFO] processed 5000/25000
[INFO] processed 6000/25000
[INFO] processed 7000/25000
[INFO] processed 8000/25000
[INFO] processed 9000/25000
[INFO] processed 10000/25000
[INFO] processed 11000/25000
[INFO] processed 12000/25000
[INFO] processed 13000/25000
[INFO] processed 14000/25000
[INFO] processed 15000/25000
[INFO] processed 16000/25000
[INFO] processed 17000/25000
[INFO] processed 18000/25000
[INFO] processed 19000/25000
[INFO] processed 20000/25000
[INFO] processed 21000/25000
[INFO] processed 22000/25000
[INFO] processed 23000/25000
[INFO] processed 24000/25000


# แบ่งเป็น 80/20

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(np.array(rawImages), np.array(labels), random_state=1)

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=1)

# แปลง output ของเราให้เป็น 1-hot

In [30]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(y_test)

y_train = le.transform(y_train)
y_test = le.transform(y_test)

y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

#X_train /= 255
#X_test /= 255

# สร้างโมเดลขึ้นมา

In [32]:
def classification_model():
     # create model
    model = Sequential()
    model.add(Dense(124, activation='relu', input_shape=(X_train.shape[1],)))
    model.add(Dropout(0.2))
    model.add(Dense(124, activation='relu'))
    model.add(Dropout(0.2))   
    model.add(Dense(32, activation='relu'))
    model.add(Dense(y_train.shape[1], activation='softmax'))
    opt = keras.optimizers.Adam(lr=0.0001, decay=1e-6)
    #opt = keras.optimizers.rmsprop(lr=0.0001, decay=1e-6)
     # compile model
    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [33]:
# build the model
model = classification_model()
# fit the model
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, verbose=2)

Train on 20000 samples, validate on 5000 samples
Epoch 1/10
 - 3s - loss: 0.6702 - acc: 0.5843 - val_loss: 0.6367 - val_acc: 0.6232
Epoch 2/10
 - 2s - loss: 0.6319 - acc: 0.6332 - val_loss: 0.6226 - val_acc: 0.6436
Epoch 3/10
 - 2s - loss: 0.6227 - acc: 0.6433 - val_loss: 0.6183 - val_acc: 0.6496
Epoch 4/10
 - 2s - loss: 0.6181 - acc: 0.6473 - val_loss: 0.6159 - val_acc: 0.6482
Epoch 5/10
 - 2s - loss: 0.6129 - acc: 0.6524 - val_loss: 0.6154 - val_acc: 0.6478
Epoch 6/10
 - 2s - loss: 0.6064 - acc: 0.6570 - val_loss: 0.6127 - val_acc: 0.6516
Epoch 7/10
 - 2s - loss: 0.6015 - acc: 0.6675 - val_loss: 0.6103 - val_acc: 0.6548
Epoch 8/10
 - 2s - loss: 0.5963 - acc: 0.6716 - val_loss: 0.6088 - val_acc: 0.6590
Epoch 9/10
 - 2s - loss: 0.5912 - acc: 0.6797 - val_loss: 0.6087 - val_acc: 0.6570
Epoch 10/10
 - 2s - loss: 0.5846 - acc: 0.6857 - val_loss: 0.6079 - val_acc: 0.6614
